In [1]:
import cv2
from matplotlib import pyplot as plt
import numpy as np
import os

def face_detectCrop(img):
    face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray,1.1,4)
    if(len(faces)>=2):
        faces = faces[1]
        faces = np.expand_dims(faces,axis = 0)
    if(len(faces)!=0):
        if(faces[0][2]<=200):
            faces = ()
    #face = faces[0]
    return faces
import sklearn
import math
sift = cv2.SIFT_create()

***1. Crop Images to Isolate for Face***

In [2]:
Real_dir = "dataset/real/"
Fake_dir = "dataset/fake/"
real_arr=[]
fake_arr=[]
real_faces=[]
fake_faces=[]
for filename in os.listdir(Real_dir):
    if(filename.endswith(".jpg")):
        sample = cv2.imread(Real_dir+filename)
        face = face_detectCrop(sample)
        #print(face)
        real_faces.append(face)
#         print(Real_dir+filename)
        real_arr.append(sample)

for filename in os.listdir(Fake_dir):
    if(filename.endswith(".jpg")):
        sample = cv2.imread(Fake_dir+filename)
        face = face_detectCrop(sample)
        fake_faces.append(face)
#         print(Fake_dir+filename)
        fake_arr.append(sample)
print(len(real_faces[0]))    
# loop the Bbox array, find out empty ones and replace it with detected ones    
for i in range(len(real_faces)-2):
    if i%3 ==0:
        if len(real_faces[i+1]) == 0:
            real_faces[i+1] = real_faces[i]
        if len(real_faces[i+2]) == 0:
            real_faces[i+2] = real_faces[i]

for i in range(len(real_faces)-2):
    if i%3 ==1:
        if len(real_faces[i+1]) == 0:
            real_faces[i+1] = real_faces[i]
        if len(real_faces[i-1]) == 0:
            real_faces[i-1] = real_faces[i]
            
for i in range(len(real_faces)-2):
    if i%3 ==2:
        if len(real_faces[i-1]) == 0:
            real_faces[i-1] = real_faces[i]
        if len(real_faces[i-2]) == 0:
            real_faces[i-2] = real_faces[i]

for i in range(len(fake_faces)-2):
    if i%3 ==0:
        if len(fake_faces[i+1]) == 0:
            fake_faces[i+1] = fake_faces[i]
        if len(fake_faces[i+2]) == 0:
            fake_faces[i+2] = fake_faces[i]

for i in range(len(fake_faces)-2):
    if i%3 ==1:
        if len(fake_faces[i+1]) == 0:
            fake_faces[i+1] = fake_faces[i]
        if len(fake_faces[i-1]) == 0:
            fake_faces[i-1] = fake_faces[i]
            
for i in range(len(fake_faces)-2):
    if i%3 ==2:
        if len(fake_faces[i-1]) == 0:
            fake_faces[i-1] = fake_faces[i]
        if len(fake_faces[i-2]) == 0:
            fake_faces[i-2] = fake_faces[i]      

# len_real = len(real_arr)
# len_fake = len(fake_arr)
# print(real_faces)
# print(fake_faces)


1


***2. Function for generating Disparity Map***

In [3]:
index_params = dict(algorithm = 1, tress = 5)
search_params = dict (checks = 50)
flann = cv2.FlannBasedMatcher(index_params,search_params)
bf=cv2.BFMatcher()
retval = cv2.StereoBM_create(numDisparities=16, blockSize=5)

def Generate_kp_and_Descriptors(img1,img2, gray1, gray2):
    kp1, des1 = sift.detectAndCompute (img1, None)
    kp2, des2 = sift.detectAndCompute (img2, None)
    
    #matches = flann.knnMatch(des1, des2,k=2)
    matches = bf.knnMatch(des1,des2,k=2)
    pts1 = []
    pts2 = []
    
    for i,(m,n) in enumerate(matches):
        if m.distance <0.9*n.distance:
            pts2.append(kp2[m.trainIdx].pt)
            pts1.append(kp1[m.queryIdx].pt)
    pts1 = np.int32(pts1)
    pts2 = np.int32(pts2)
    F, mask = cv2.findFundamentalMat(pts1,pts2,cv2.FM_LMEDS)
    
    pts1 = pts1[mask.ravel()==1]
    pts2 = pts2[mask.ravel()==1]
    
    size1 = (img1.shape[1], img1.shape[0])
    size2 = (img2.shape[1], img2.shape[0])
    
    retval, H1, H2 = cv2.stereoRectifyUncalibrated(pts1, pts2, F, size1)
    output1 = cv2.warpPerspective(gray1, H1, size1, flags = cv2.INTER_LINEAR)
    output2 = cv2.warpPerspective(gray2, H2, size1, flags = cv2.INTER_LINEAR)
    
    #print(output1.shape)
    #print(output2.shape)
    
    retval = cv2.StereoBM_create(numDisparities=16, blockSize=5)
    disparity_output = retval.compute(output1, output2)
    return disparity_output

***3. Calculate Standard Deviation Between Disparity Maps and Calculate for possible Classification outcomes***

In [4]:
stand_devReal1=[]
stand_devReal2=[]
stand_devReal3=[]

stand_devFake1=[]
stand_devFake2=[]
stand_devFake3=[]

i=0

for arr in fake_arr:
#     print(arr.shape)
    i+=1
# print("real_arr count:", i)

for i in range(len(real_faces)-2):
    if i%3 == 0:
        #crop0 = i+0, crop1 = i+1, crop2 = i+2
#         print(real_arr[i].shape)
#         print(real_faces[i].shape)
        crop0 = real_arr[i][real_faces[i][0][1]:real_faces[i][0][1]+real_faces[i][0][3], real_faces[i][0][0]:real_faces[i][0][0]+real_faces[i][0][2],:]
        crop1 = real_arr[i+1][real_faces[i+1][0][1]:real_faces[i+1][0][1]+real_faces[i+1][0][3], real_faces[i+1][0][0]:real_faces[i+1][0][0]+real_faces[i+1][0][2],:]
        crop2 = real_arr[i+2][real_faces[i+2][0][1]:real_faces[i+2][0][1]+real_faces[i+2][0][3], real_faces[i+2][0][0]:real_faces[i+2][0][0]+real_faces[i+2][0][2],:]
        #plt.imshow(crop0)
        #plt.show()
        #plt.imshow(crop1)
#         plt.show()
#         plt.imshow(crop2)
#         plt.show()
#         print(i)
        disparity1 = Generate_kp_and_Descriptors(crop1,crop2,cv2.cvtColor(crop1,cv2.COLOR_BGR2GRAY),cv2.cvtColor(crop2,cv2.COLOR_BGR2GRAY))
        disparity2 = Generate_kp_and_Descriptors(crop0,crop1,cv2.cvtColor(crop0,cv2.COLOR_BGR2GRAY),cv2.cvtColor(crop1,cv2.COLOR_BGR2GRAY))
        disparity3 = Generate_kp_and_Descriptors(crop0,crop2,cv2.cvtColor(crop0,cv2.COLOR_BGR2GRAY),cv2.cvtColor(crop2,cv2.COLOR_BGR2GRAY))
        stand_devReal1.append(np.std(disparity1))
        stand_devReal2.append(np.std(disparity2))
        stand_devReal3.append(np.std(disparity3))


for i in range(len(fake_faces)):
    if i%3 == 0:
        crop0 = fake_arr[i][fake_faces[i][0][1]:fake_faces[i][0][1]+fake_faces[i][0][3], fake_faces[i][0][0]:fake_faces[i][0][0]+fake_faces[i][0][2]]
        crop1 = fake_arr[i+1][fake_faces[i+1][0][1]:fake_faces[i+1][0][1]+fake_faces[i+1][0][3], fake_faces[i+1][0][0]:fake_faces[i+1][0][0]+fake_faces[i+1][0][2]]
        crop2 = fake_arr[i+2][fake_faces[i+2][0][1]:fake_faces[i+2][0][1]+fake_faces[i+2][0][3], fake_faces[i+2][0][0]:fake_faces[i+2][0][0]+fake_faces[i+2][0][2]]
        disparity1 = Generate_kp_and_Descriptors(crop1,crop2,cv2.cvtColor(crop1,cv2.COLOR_BGR2GRAY),cv2.cvtColor(crop2,cv2.COLOR_BGR2GRAY))
        disparity2 = Generate_kp_and_Descriptors(crop0,crop1,cv2.cvtColor(crop0,cv2.COLOR_BGR2GRAY),cv2.cvtColor(crop1,cv2.COLOR_BGR2GRAY))
        disparity3 = Generate_kp_and_Descriptors(crop0,crop2,cv2.cvtColor(crop0,cv2.COLOR_BGR2GRAY),cv2.cvtColor(crop2,cv2.COLOR_BGR2GRAY))
        stand_devFake1.append(np.std(disparity1))
        stand_devFake2.append(np.std(disparity2))
        stand_devFake3.append(np.std(disparity3))
# print(stand_devReal1)
# print(stand_devReal2)
# print(stand_devReal3)
stand_devReal1 = np.asarray(stand_devReal1)
stand_devReal2 = np.asarray(stand_devReal2)
stand_devReal3 = np.asarray(stand_devReal3)
sum_real = stand_devReal1+stand_devReal2+stand_devReal3
print("Real:",stand_devReal1+stand_devReal2+stand_devReal3)
print('_______________')
# print(stand_devFake1)
# print(stand_devFake2)
# print(stand_devFake3)
stand_devFake1 = np.asarray(stand_devFake1)
stand_devFake2 =np.asarray(stand_devFake2)
stand_devFake3 =np.asarray(stand_devFake3)
print("Fake:",stand_devFake1+stand_devFake2+stand_devFake3)
sum_fake = stand_devFake1+stand_devFake2+stand_devFake3
tp = 0
fp = 0
tn = 0
fn = 0
for i in range(stand_devReal1.shape[0]):
    if(sum_real[i]>sum_fake[i]):
        #TP
        tp+=1
        tn+=1
    else:
        fp +=1
        fn+=1
Precision = 0
Precision = tp/(tp+fp)
Recall = 0
Recall = tp/(tp+fn)
print("Confusion Matrix:")
print("Num of TN:", tn)
print("Num of FP:", fp)
print("Num of FN:", fn)
print("Num of TP:", tp)
print('Precision:',Precision)
print('Recall:', Recall)

Real: [206.54342915 235.73028699 192.34546776 173.90973532 176.20855297
 215.97316717 193.59694583 240.76589933 249.95705775 133.26038501
 198.0713639  137.50392262]
_______________
Fake: [161.26400083 202.69327166 159.20618755 230.92805916 256.34033732
 234.4691242  216.11523897 135.66858387 187.87558214 175.44079272
 192.2689381  229.07029547]
Confusion Matrix:
Num of TN: 6
Num of FP: 6
Num of FN: 6
Num of TP: 6
Precision: 0.5
Recall: 0.5
